In [ ]:
!pip3 install fairseq
!pip3 install fastbpe
!pip3 install vncorenlp
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
!tar -xzvf PhoBERT_base_fairseq.tar.gz

--2023-05-21 14:22:29--  https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
Resolving public.vinai.io (public.vinai.io)... 13.227.219.99, 13.227.219.93, 13.227.219.12, ...
Connecting to public.vinai.io (public.vinai.io)|13.227.219.99|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1243308020 (1.2G) [application/x-tar]
Saving to: ‘PhoBERT_base_fairseq.tar.gz.1’

PhoBERT_base_fairse 100%[===================>]   1.16G   228MB/s    in 4.9s    

2023-05-21 14:22:35 (241 MB/s) - ‘PhoBERT_base_fairseq.tar.gz.1’ saved [1243308020/1243308020]

PhoBERT_base_fairseq/
PhoBERT_base_fairseq/bpe.codes
PhoBERT_base_fairseq/model.pt
PhoBERT_base_fairseq/dict.txt


In [ ]:
!ls PhoBERT_base_fairseq

bpe.codes  dict.txt  model.pt


In [ ]:
# Load the model in fairseq
from fairseq.models.roberta import RobertaModel
phoBERT = RobertaModel.from_pretrained('PhoBERT_base_fairseq', checkpoint_file='model.pt')
phoBERT.eval()  # disable dropout (or leave in train mode to finetune

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(64001, 768, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(258, 768, padding_idx=1)
        (layernorm_embedding): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0-11): 12 x TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

In [ ]:
from fairseq.data.encoders.fastbpe import fastBPE

class BPE():
  bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'

args = BPE()
phoBERT.bpe = fastBPE(args)
tokens = phoBERT.encode('Trường đại học khoa học tự nhiên Hà Nội')
print('tokens list : ', tokens)
# Decode ngược lại thành câu từ chuỗi index token
phoBERT.decode(tokens)  # 'Hello world!'

tokens list :  tensor([   0,  622, 2919,  222, 2054,  222,  385, 8249, 1696, 7657,    2])


'Trường đại học khoa học tự nhiên Hà Nội'

In [ ]:
# Trích xuất đặt trưng từ lớp cuối cùng
last_layer_features = phoBERT.extract_features(tokens)

print('token size: ', tokens.size())
print('size of last layer: ', last_layer_features.size())

# Extract all layer's features (layer 0 is the embedding layer)
all_layers = phoBERT.extract_features(tokens, return_all_hiddens=True)
print('number layer in all layers: ', len(all_layers))

# Ma trận vector đặc trưng của layer cuối bằng ma trận vector đặc trưng lấy ra
# từ all layer
print('Last layer features: ', all_layers[-1] == last_layer_features)

token size:  torch.Size([11])
size of last layer:  torch.Size([1, 11, 768])
number layer in all layers:  13
Last layer features:  tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]])


In [ ]:
# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter)
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

--2023-05-21 14:23:12--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M   173MB/s    in 0.2s    

2023-05-21 14:23:12 (173 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2023-05-21 14:23:12--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting respon

In [ ]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

In [ ]:
from tqdm import tqdm
import torch
from fairseq.data import Dictionary
import numpy as np

In [ ]:
max_sequence_length = 256
def convert_lines(lines, vocab, bpe):
  '''
  lines: list các văn bản input
  vocab: từ điển dùng để encoding subwords
  bpe:
  '''
  # Khởi tạo ma trận output
  outputs = np.zeros((len(lines), max_sequence_length), dtype=np.int32) # --> shape (number_lines, max_seq_len)
  # Index của các token cls (đầu câu), eos (cuối câu), padding (padding token)
  cls_id = 0
  eos_id = 2
  pad_id = 1

  for idx, row in tqdm(enumerate(lines), total=len(lines)):
    # Mã hóa subwords theo byte pair encoding(bpe)
    subwords = bpe.encode('<s> '+ row +' </s>')
    input_ids = vocab.encode_line(subwords, append_eos=False, add_if_not_exist=False).long().tolist()
    # Truncate input nếu độ dài vượt quá max_seq_len
    if len(input_ids) > max_sequence_length:
      input_ids = input_ids[:max_sequence_length]
      input_ids[-1] = eos_id
    else:
      # Padding nếu độ dài câu chưa bằng max_seq_len
      input_ids = input_ids + [pad_id, ]*(max_sequence_length - len(input_ids))

    outputs[idx,:] = np.array(input_ids)
  return outputs

# Load the dictionary
vocab = Dictionary()
vocab.add_from_file("/content/PhoBERT_base_fairseq/dict.txt")



# Test encode lines
lines = ['Học_sinh được nghỉ học bắt dầu từ tháng 3 để tránh dịch covid-19', 'số lượng ca nhiễm bệnh đã giảm bắt đầu từ tháng 5 nhờ biện pháp mạnh tay']
[x1, x2] = convert_lines(lines, vocab, phoBERT.bpe)
print('x1 tensor encode: {}, shape: {}'.format(x1[:10], x1.size))
print('x1 tensor decode: ', phoBERT.decode(torch.tensor(x1))[:103])

100%|██████████| 2/2 [00:00<00:00, 761.01it/s]

x1 tensor encode: [63117  1302   884  5958    11   915   222   537   933    39], shape: 256
x1 tensor decode:  <s> Học_sinh được nghỉ học bắt dầu từ tháng 3 để tránh dịch covid-19 </s> <pad> <pad> <pad> <pad> <pad>


In [ ]:
import json
import chardet

In [ ]:
import json
from sklearn.model_selection import train_test_split

# Load data from text file
with open('data.json', 'r') as f:
    data = [json.loads(line) for line in f]

# Split data into train and test
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Save train data to JSON file
with open('train.json', 'w') as f:
    for item in train_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

# Save test data to JSON file
with open('test.json', 'w') as f:
    for item in test_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

In [ ]:
# Đọc lại file với encoding tương ứng
with open('train.json' ,'r') as f:
    train = f.read()
# Chuyển nội dung thành danh sách các đối tượng
# json.loads(và json.load) không giải mã nhiều đối tượng json.
# Kết xuất JSON của data.json có một đối tượng trên mỗi dòng
objects = [json.loads(line) for line in open('train.json', 'r')]

# Lấy giá trị của thuộc tính sentence trong từng đối tượng
sentences = []
for obj in objects:
    sentence = obj['sentence']
    sentences.append(sentence)

labels = []
for obj in objects:
    label = obj['labels']
    labels.append(label)

# In ra 5 giá trị đầu của sentence và label
print(sentences[:5])
print(labels[:5])

['công tác quản lý xử lý vi phạm hành chính xlvphc và theo dõi thi hành pháp luật được đẩy mạnh thực hiện , đảm bảo sự gắn kết chặt chẽ giữa thi hành pháp luật với xây dựn pháp luật .', 'Tuy nhiên, vì lý do cá nhân nên cha của Linh cũng không chuộc được xe.', 'google cũng yêu cầu tòa giữ kín một số thông tin thiệt hại mà google tiết lộ trong một đơn kiện gửi tòa .', 'Cháu đòi tiền cơm, dì đòi tiền nhà.', 'Luật sư VŨ PHI LONG , nguyên Phó Chánh Tòa Hình sự TAND TP.HCM PHƯƠNG LOAN']
[False, True, False, True, True]


In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
lb.fit(labels)
y = lb.fit_transform(labels)
print(lb.classes_)
# print('Top 5 classes indices: ', y[:5])
print(y[:5])

[False  True]
[0 1 0 1 1]


In [ ]:
X = sentences

In [ ]:
X = convert_lines(X, vocab, phoBERT.bpe)

100%|██████████| 12000/12000 [00:05<00:00, 2273.01it/s]


In [ ]:
phoBERT_cls = RobertaModel.from_pretrained('PhoBERT_base_fairseq', checkpoint_file='model.pt')
phoBERT_cls.eval()  # disable dropout (or leave in train mode to finetune
phoBERT_cls.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT

In [ ]:
phoBERT_cls.register_classification_head('new_task', num_classes=2)
tokens = sentences[0]
token_idxs = phoBERT_cls.encode(tokens)
logprobs = phoBERT_cls.predict('new_task', token_idxs)  # tensor([[-1.1050, -1.0672, -1.1245]], grad_fn=<LogSoftmaxBackward>)
logprobs

tensor([[-0.7590, -0.6314]], grad_fn=<LogSoftmaxBackward0>)

In [ ]:
import numpy as np
with torch.no_grad():
  x = np.argmax(logprobs, axis = 1)
  print(x)

tensor([1])


In [ ]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def evaluate(logits, targets):
    """
    Đánh giá model sử dụng accuracy và f1 scores.
    Args:
        logits (B,C): torch.LongTensor. giá trị predicted logit cho class output.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        acc (float): the accuracy score
        f1 (float): the f1 score
    """
    # Tính accuracy score và f1_score
    logits = logits.detach().cpu().numpy()
    y_pred = np.argmax(logits, axis = 1)
    targets = targets.detach().cpu().numpy()
    f1 = f1_score(targets, y_pred, average='weighted')
    acc = accuracy_score(targets, y_pred)
    return acc, f1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logits = torch.tensor([[0.1, 0.2, 0.7],
                       [0.4, 0.1, 0.5],
                       [0.1, 0.2, 0.7]]).to(device)
targets = torch.tensor([0, 1, 1]).to(device)
evaluate(logits, targets)

(0.0, 0.0)

In [ ]:
def validate(valid_loader, model, device):
    model.eval()
    accs = []
    f1s = []
    with torch.no_grad():
        for x_batch, y_batch in valid_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model.predict('new_task', x_batch)
            logits = torch.exp(outputs)
            acc, f1 = evaluate(logits, y_batch)
            accs.append(acc)
            f1s.append(f1)

    mean_acc = np.mean(accs)
    mean_f1 = np.mean(f1s)
    return mean_acc, mean_f1

In [ ]:
import time, os

In [ ]:
def trainOnEpoch(train_loader, model, optimizer, epoch, num_epochs, criteria, device, log_aggr = 100):
    model.train()
    sum_epoch_loss = 0
    sum_acc = 0
    sum_f1 = 0
    start = time.time()
    for i, (x_batch, y_batch) in enumerate(train_loader):
      x_batch = x_batch.to(device)
      y_batch = y_batch.to(device)
      optimizer.zero_grad()
      y_pred = model.predict('new_task', x_batch)
      logits = torch.exp(y_pred)
      acc, f1 = evaluate(logits, y_batch)
      loss = criteria(y_pred, y_batch)
      loss.backward()
      optimizer.step()

      loss_val = loss.item()
      sum_epoch_loss += loss_val
      sum_acc += acc
      sum_f1 += f1
      iter_num = epoch * len(train_loader) + i + 1

      if i % log_aggr == 0:
            print('[TRAIN] epoch %d/%d  observation %d/%d batch loss: %.4f (avg %.4f),  avg acc: %.4f, avg f1: %.4f, (%.2f im/s)'
                % (epoch + 1, num_epochs, i, len(train_loader), loss_val, sum_epoch_loss / (i + 1),  sum_acc/(i+1), sum_f1/(i+1),
                  len(x_batch) / (time.time() - start)))
      start = time.time()


In [ ]:
from sklearn.model_selection import StratifiedKFold
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.autograd import Variable
from torch.backends import cudnn

In [ ]:
from transformers.modeling_utils import *
from transformers import *

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memorry_efficient_attenti

In [ ]:
!ls -la --block-size=M PhoBERT_base_fairseq/

total 1291M
drwxrwxr-x 2 1017 1025    1M Mar  2  2020 .
drwxr-xr-x 1 root root    1M May 19 20:21 ..
-rw-rw-r-- 1 1017 1025    2M Mar  2  2020 bpe.codes
-rw-rw-r-- 1 1017 1025    1M Mar  2  2020 dict.txt
-rw-rw-r-- 1 1017 1025 1289M Jan 20  2020 model.pt


In [ ]:
EPOCHS = 20
BATCH_SIZE = 6
ACCUMULATION_STEPS = 5
FOLD = 4
LR = 0.0001
LR_DC_STEP = 80
LR_DC = 0.1
CUR_DIR = os.path.dirname(os.getcwd())
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
FOLD = 4
CKPT_PATH2 = '/content/model_ckpt2'

if not os.path.exists(CKPT_PATH2):
    os.mkdir(CKPT_PATH2)

# Khởi tạo DataLoader
splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(X, y))

for fold, (train_idx, val_idx) in enumerate(splits):
    best_score = 0
    if fold != FOLD:
        continue
    print("Training for fold {}".format(fold))

    # Create dataset
    train_dataset = torch.utils.data.TensorDataset(torch.tensor(X[train_idx],dtype=torch.long), torch.tensor(y[train_idx],dtype=torch.long))
    valid_dataset = torch.utils.data.TensorDataset(torch.tensor(X[val_idx],dtype=torch.long), torch.tensor(y[val_idx],dtype=torch.long))

    # Create DataLoader
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Khởi tạo model:
    MODEL_LAST_CKPT = os.path.join(CKPT_PATH2, 'latest_checkpoint.pth.tar')
    if os.path.exists(MODEL_LAST_CKPT):
      print('Load checkpoint model!')
      checkpoint = torch.load(MODEL_LAST_CKPT)
      state_dict = checkpoint['state_dict']

      # Xác định các khóa không mong đợi
      unexpected_keys = []
      for key in state_dict.keys():
          if 'classification_heads.new_task' in key:
              unexpected_keys.append(key)

      # Loại bỏ các khóa không mong đợi
      for key in unexpected_keys:
          del state_dict[key]

      # Tải lại state_dict vào phoBERT_cls_temp
      phoBERT_cls.load_state_dict(state_dict)

    else:
      print('Load model pretrained!')
      # Load the model in fairseq
      from fairseq.models.roberta import RobertaModel
      from fairseq.data.encoders.fastbpe import fastBPE
      from fairseq.data import Dictionary

      phoBERT_cls = RobertaModel.from_pretrained('PhoBERT_base_fairseq', checkpoint_file='model.pt')
      phoBERT_cls.eval()  # disable dropout (or leave in train mode to finetune

      # # Load BPE
      # class BPE():
      #   bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'

      # args = BPE()
      # phoBERT_cls.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT

      # Add header cho classification với số lượng classes = 10
      phoBERT_cls.register_classification_head('new_task', num_classes=2)

    ## Load BPE
    print('Load BPE')
    class BPE():
      bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'

    args = BPE()
    phoBERT_cls.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT
    phoBERT_cls.to(DEVICE)

    # Khởi tạo optimizer và scheduler, criteria
    print('Init Optimizer, scheduler, criteria')
    param_optimizer = list(phoBERT_cls.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    num_train_optimization_steps = int(EPOCHS*len(train_dataset)/BATCH_SIZE/ACCUMULATION_STEPS)
    optimizer = AdamW(optimizer_grouped_parameters, lr=LR, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=num_train_optimization_steps)  # scheduler với linear warmup
    scheduler0 = get_constant_schedule(optimizer)  # scheduler với hằng số
    # optimizer = optim.Adam(phoBERT_cls.parameters(), LR)
    criteria = nn.NLLLoss()
    # scheduler = StepLR(optimizer, step_size = LR_DC_STEP, gamma = LR_DC)
    avg_loss = 0.
    avg_accuracy = 0.
    frozen = True
    for epoch in tqdm(range(EPOCHS)):
        # warm up tại epoch đầu tiên, sau epoch đầu sẽ phá băng các layers
        if epoch > 0 and frozen:
            for child in phoBERT_cls.children():
                for param in child.parameters():
                    param.requires_grad = True
            frozen = False
            del scheduler0
            torch.cuda.empty_cache()
        # Train model on EPOCH
        print('Epoch: ', epoch)
        trainOnEpoch(train_loader=train_loader, model=phoBERT_cls, optimizer=optimizer, epoch=epoch, num_epochs=EPOCHS, criteria=criteria, device=DEVICE, log_aggr=100)
        # scheduler.step(epoch = epoch)
        # Phá băng layers sau epoch đầu tiên
        if not frozen:
            scheduler.step()
        else:
            scheduler0.step()
        optimizer.zero_grad()
        # Validate on validation set
        acc, f1 = validate(valid_loader, phoBERT_cls, device=DEVICE)
        print('Epoch {} validation: acc: {:.4f}, f1: {:.4f} \n'.format(epoch, acc, f1))

        # Store best model checkpoint
        ckpt_dict = {
            'epoch': epoch + 1,
            'state_dict': phoBERT_cls.state_dict(),
            'optimizer': optimizer.state_dict()
        }
        # Save model checkpoint into 'latest_checkpoint.pth.tar'
        torch.save(ckpt_dict, MODEL_LAST_CKPT)
        #for key in phoBERT_cls.state_dict():
          #print(key)

Training for fold 4
Load model pretrained!
Load BPE


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Init Optimizer, scheduler, criteria


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch:  0
[TRAIN] epoch 1/20  observation 0/1600 batch loss: 0.6798 (avg 0.6798),  avg acc: 0.6667, avg f1: 0.6667, (1.81 im/s)
[TRAIN] epoch 1/20  observation 100/1600 batch loss: 0.3961 (avg 0.7116),  avg acc: 0.5891, avg f1: 0.4786, (23.34 im/s)
[TRAIN] epoch 1/20  observation 200/1600 batch loss: 0.5637 (avg 0.6677),  avg acc: 0.6468, avg f1: 0.5376, (22.92 im/s)
[TRAIN] epoch 1/20  observation 300/1600 batch loss: 0.6369 (avg 0.6618),  avg acc: 0.6523, avg f1: 0.5417, (22.32 im/s)
[TRAIN] epoch 1/20  observation 400/1600 batch loss: 0.5196 (avg 0.6546),  avg acc: 0.6584, avg f1: 0.5468, (22.33 im/s)
[TRAIN] epoch 1/20  observation 500/1600 batch loss: 0.7310 (avg 0.6554),  avg acc: 0.6564, avg f1: 0.5429, (22.69 im/s)
[TRAIN] epoch 1/20  observation 600/1600 batch loss: 0.5169 (avg 0.6508),  avg acc: 0.6611, avg f1: 0.5480, (22.51 im/s)
[TRAIN] epoch 1/20  observation 700/1600 batch loss: 0.8780 (avg 0.6488),  avg acc: 0.6626, avg f1: 0.5489, (22.58 im/s)
[TRAIN] epoch 1/20  obser

...

 90%|█████████ | 18/20 [2:21:15<15:39, 469.89s/it]

Epoch:  18
[TRAIN] epoch 19/20  observation 0/1600 batch loss: 0.6424 (avg 0.6424),  avg acc: 0.6667, avg f1: 0.5333, (17.84 im/s)
[TRAIN] epoch 19/20  observation 100/1600 batch loss: 0.7400 (avg 0.6520),  avg acc: 0.6436, avg f1: 0.5278, (22.50 im/s)
[TRAIN] epoch 19/20  observation 200/1600 batch loss: 0.6401 (avg 0.6528),  avg acc: 0.6426, avg f1: 0.5227, (22.46 im/s)
[TRAIN] epoch 19/20  observation 300/1600 batch loss: 0.6380 (avg 0.6567),  avg acc: 0.6357, avg f1: 0.5143, (22.81 im/s)
[TRAIN] epoch 19/20  observation 400/1600 batch loss: 0.6428 (avg 0.6479),  avg acc: 0.6505, avg f1: 0.5328, (22.41 im/s)
[TRAIN] epoch 19/20  observation 500/1600 batch loss: 0.8893 (avg 0.6429),  avg acc: 0.6577, avg f1: 0.5409, (22.35 im/s)
[TRAIN] epoch 19/20  observation 600/1600 batch loss: 0.8920 (avg 0.6386),  avg acc: 0.6636, avg f1: 0.5479, (22.58 im/s)
[TRAIN] epoch 19/20  observation 700/1600 batch loss: 1.0220 (avg 0.6376),  avg acc: 0.6652, avg f1: 0.5498, (22.30 im/s)
[TRAIN] epoch 1

 95%|█████████▌| 19/20 [2:29:07<07:50, 470.44s/it]

Epoch:  19
[TRAIN] epoch 20/20  observation 0/1600 batch loss: 0.3671 (avg 0.3671),  avg acc: 1.0000, avg f1: 1.0000, (16.63 im/s)
[TRAIN] epoch 20/20  observation 100/1600 batch loss: 0.5215 (avg 0.6382),  avg acc: 0.6667, avg f1: 0.5519, (22.09 im/s)
[TRAIN] epoch 20/20  observation 200/1600 batch loss: 0.3971 (avg 0.6380),  avg acc: 0.6667, avg f1: 0.5528, (22.27 im/s)
[TRAIN] epoch 20/20  observation 300/1600 batch loss: 0.6380 (avg 0.6355),  avg acc: 0.6694, avg f1: 0.5560, (22.75 im/s)
[TRAIN] epoch 20/20  observation 400/1600 batch loss: 0.4132 (avg 0.6387),  avg acc: 0.6646, avg f1: 0.5488, (22.41 im/s)
[TRAIN] epoch 20/20  observation 500/1600 batch loss: 0.5188 (avg 0.6354),  avg acc: 0.6693, avg f1: 0.5539, (22.50 im/s)
[TRAIN] epoch 20/20  observation 600/1600 batch loss: 0.3980 (avg 0.6346),  avg acc: 0.6705, avg f1: 0.5554, (22.29 im/s)
[TRAIN] epoch 20/20  observation 700/1600 batch loss: 0.6353 (avg 0.6331),  avg acc: 0.6726, avg f1: 0.5584, (22.69 im/s)
[TRAIN] epoch 2

100%|██████████| 20/20 [2:36:56<00:00, 470.84s/it]


In [ ]:
import json
import torch
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Đường dẫn đến tệp dữ liệu test
test_file = "s-data.json"

# Load test data
with open(test_file, "r") as f:
    test_data = [json.loads(line) for line in f]

phoBERT_cls.register_classification_head('new_task', num_classes=2)
phoBERT_cls = phoBERT_cls.to('cuda')

true_labels = []
pred_labels = []
convert_text_int = []
for example in test_data:
    text = example['sentence']
    label = example['labels']

    encoded = convert_lines(text, vocab, phoBERT.bpe)  # Sử dụng [text] thay vì text
    convert_text_int.append(encoded[0])  # Thêm mã hóa của từng văn bản vào danh sách

my_tensor = torch.tensor(convert_text_int)
my_tensor = my_tensor.to('cuda')

with torch.no_grad():
    logits = phoBERT_cls.predict('new_task', my_tensor)

# Get predicted labels
pred_labels = torch.argmax(logits, dim=1).tolist()

# Calculate evaluation metrics
true_labels = [example['labels'] for example in test_data]
accuracy = accuracy_score(true_labels, pred_labels)
recall = recall_score(true_labels, pred_labels, average='macro')
precision = precision_score(true_labels, pred_labels, average='macro')
f1 = f1_score(true_labels, pred_labels, average='macro')

# Print evaluation metrics
print('Accuracy:', accuracy)
print('Recall:', recall)
print('Precision:', precision)
print('F1:', f1)

100%|██████████| 91/91 [00:00<00:00, 7521.86it/s]


Accuracy: 0.8043912175648703
Recall: 0.5623725055432373
Precision: 0.5445123109057536
F1: 0.5490300536409729


In [ ]:
import torch
import numpy as np

# Chuỗi câu bạn muốn dự đoán
text = sentences[0]

# Tiền xử lý dữ liệu: encoded là đầu vào được mã hóa của câu
encoded = phoBERT.encode(text)

# Chuyển đổi dữ liệu thành tensor PyTorch
input_tensor = torch.tensor(encoded)

# Đảm bảo rằng mô hình và dữ liệu đều ở chế độ "eval" (đánh giá)
phoBERT_cls.eval()

# Tính toán dự đoán với torch.no_grad() để tắt tính toán gradient
with torch.no_grad():
    # Truyền dữ liệu qua mô hình để nhận logits
    logits = phoBERT_cls.predict('new_task', input_tensor)
import numpy as np
with torch.no_grad():
  x = np.argmax(logits, axis = 1)
  print(x)